In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# Phantom Array 🛰️📡👻📡🛰️

## 🎯 Learning goals

By the end of this mini‑lesson, you can:
- Describe how motion creates a **synthetic aperture** (a “phantom array”) for fine **azimuth resolution**.
- Predict how **aperture length** affects image sharpness along **azimuth**.

# Sharpening Our Images by Building a Virtual Array

## 🛰️ Synthesizing an Aperture  

We’re almost there! 🔥 You already know how to get **high range resolution** with chirped signals. And you’ve seen that **high azimuth resolution** normally demands a *huge* antenna.  

Enter **synthetic aperture radar (SAR)** — a clever way to “fake” a giant antenna without actually building one.  

Instead of one massive antenna, we **move our radar** along a path, transmitting a pulse and recording echoes at each position. Step by step 🚶‍♂️📡, these signals combine into the effect of a much larger aperture.  

It’s the same basic idea as **SLAR** from a previous notebook. But now, by recording *all* those echoes in detail and applying some clever signal processing, we can pull off something remarkable.  

Before we jump into the magic, let’s first see what the raw data looks like.  

## 🗄️ How SAR Data Is Collected  

Picture a satellite with its antenna locked on a **single fixed point** on the ground (**spotlight mode** imaging). 🔦  

As the satellite glides forward along its orbit, the same simple cycle repeats:  

1️⃣ **Transmit** a chirped radar pulse.  
2️⃣ The pulse **travels to Earth**, bounces off the surface, and **returns**.  
3️⃣ **Record** the echoes in memory.  
4️⃣ **Move forward** and repeat.  

🔇 **Why don’t we listen while we transmit?**  
Because the transmitter is *blazing* powerful, and the receiver is *super* sensitive. Listening during transmission would be like trying to catch a whisper while someone blasts a megaphone in your ear. 📢 **Not fun—and possibly destructive** to the electronics.  

So we **transmit first**, then **listen for the echoes**—never both at once.  

And because the satellite is **far** from Earth’s surface, there’s a built-in delay before useful echoes arrive. We only record the portion of the signal that contains reflections from our area of interest, ignoring everything else. This keeps **data collection lean** and **processing efficient**. 

---

## 🎥 Animation: SAR Imaging Geometry  

In this animation, you can see how SAR data is collected:  
- 🛰️ A satellite glides smoothly along its orbit.  
- 📡 Its antenna beam is **continuously steered** to illuminate the same target area on the ground.  
- 🦸‍♂️ Our caped crusader friend has left us a secret message, arranging shiny metal spheres into an intriguing pattern that we’ll try to resolve.  
- Each time the radar transmits a pulse and listens for echoes, a blue cross marks the spot along the trajectory. These crosses act like the **virtual elements** of a giant *phantom* antenna array stretched across the satellite’s entire flight path.  

Later in processing, we combine all these measurements just as if they came from a **real phased array as long as the satellite’s entire flight path**. The result is a very narrow effective beam, giving us the fine azimuth resolution that makes SAR imaging possible.  ✨
 

In [ ]:
def build_spotlight_with_bat_outline_fixed_app():
    """
    Voilà-friendly SPOTLIGHT visualization with Bat outline on the ground.
    Uses server-side Matplotlib rendering to PNG bytes (fast for browsers).
    - Fixed collection duration: 10 s
    - Legend shows: Radar, Virtual elements, Trajectory, Beam center/contour (Bat outline not in legend)
    - Idempotent frame drawing (no accumulation -> clean replays & scrubbing)
    """
    import io
    import os
    from datetime import datetime
    import math
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import Play, IntSlider, HBox, VBox, Button, Layout, Label, jslink, Image as WImage, Dropdown
    from IPython.display import display

    # ---------- Constants ----------
    SATELLITE_VELOCITY_KMS = 7.5
    ALTITUDE_KM            = 500.0
    INCIDENCE_DEG          = 30.0
    BEAM_HALF_ANGLE_DEG    = 1.5
    FPS                    = 12
    DURATION_S             = 10.0
    FRAMES                 = int(DURATION_S * FPS)
    ELEMENT_SPACING        = 5     # frames between virtual elements
    DPI                    = 96
    IMG_WIDTH_PX           = 900   # frontend display width (px)
    ground_point           = (0.0, 0.0, 0.0)

    # ---------- Bat outline (no label so it stays out of legend) ----------
    def _make_bat_points():
        X = np.zeros((0))
        Y = np.arange(-4, 4, 0.1)
        for y in Y:
            X = np.append(
                X,
                abs(y/2) - 0.09137*y**2 + math.sqrt(1 - (abs(abs(y) - 2) - 1)**2) - 3
            )
        Y1 = np.append(np.arange(-7, -3, 0.1), np.arange(3, 7, 0.1))
        X1 = [3*math.sqrt(1 - (yy/7)**2) for yy in Y1]
        X = np.append(X, X1); Y = np.append(Y, Y1)

        Y1 = np.append(np.arange(-7, -4, 0.1), np.arange(4, 7, 0.1))
        X1 = [-3*math.sqrt(1 - (yy/7)**2) for yy in Y1]
        X = np.append(X, X1); Y = np.append(Y, Y1)

        Y1 = np.append(np.arange(-1, -0.8, 0.1), np.arange(0.8, 1, 0.1))
        X1 = [9 - 8*abs(yy) for yy in Y1]
        X = np.append(X, X1); Y = np.append(Y, Y1)

        Y1 = np.arange(-0.5, 0.5, 0.1)
        X1 = [2 for _ in Y1]
        X = np.append(X, X1); Y = np.append(Y, Y1)

        Y1 = np.append(np.arange(-2.9, -1, 0.1), np.arange(1, 2.9, 0.1))
        X1 = [1.5 - 0.5*abs(yy) - 1.89736*(math.sqrt(3 - yy**2 + 2*abs(yy)) - 2) for yy in Y1]
        X = np.append(X, X1); Y = np.append(Y, Y1)

        Y1 = np.append(np.arange(-0.7, -0.45, 0.05), np.arange(0.45, 0.7, 0.05))
        X1 = [3*abs(yy)+0.75 for yy in Y1]
        X = np.append(X, X1); Y = np.append(Y, Y1)

        X = -X
        scale = 4.0
        X, Y = X*scale, Y*scale
        Z = np.zeros_like(X)
        return X, 2*Y, Z

    bat_x, bat_y, bat_z = _make_bat_points()

    # ---------- Geometry ----------
    aperture_len = SATELLITE_VELOCITY_KMS * DURATION_S
    xs = np.linspace(-aperture_len/2, aperture_len/2, FRAMES)
    y_pos = ALTITUDE_KM * np.tan(np.radians(INCIDENCE_DEG))
    ys = np.full_like(xs, y_pos)
    zs = np.full_like(xs, ALTITUDE_KM)

    # Axis limits (as in your working spotlight app)
    max_ap_len = SATELLITE_VELOCITY_KMS * 10.0
    x_start_max = -max_ap_len / 2.0
    x_end_max   =  max_ap_len / 2.0
    mx   = 0.1 * (x_end_max - x_start_max + 1.0)
    xlim = (x_start_max - mx, x_end_max + mx)
    ylim = (-0.2 * (abs(y_pos) + 1.0), y_pos + abs(y_pos) * 0.2 + 10.0)
    zlim = (0.0, ALTITUDE_KM * 1.1 + 10.0)

    # ---------- Figure (server-side only) + front-end Image widget ----------
    # Keep the same ~aspect as before
    fig = plt.figure(figsize=(9, 6), dpi=DPI)
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title("SPOTLIGHT SAR collection (fixed 10 s)")
    ax.set_xlabel("X (km) — azimuth")
    ax.set_ylabel("Y (km) — ground range")
    ax.set_zlabel("Z (km) — altitude")
    ax.set_xlim(*xlim); ax.set_ylim(*ylim); ax.set_zlim(*zlim)

    # Static path + scene center
    ax.plot(xs, ys, zs, lw=1.0, color='0.85')
    ax.plot([0],[0],[0], marker='o', markersize=6, color='k')

    # Bat outline (no legend entry)
    ax.scatter(bat_x, bat_y, bat_z, marker='x', s=4, alpha=0.9,
               color='black', depthshade=False)

    # Dynamic artists (with labels for legend)
    los_line   = ax.plot([],[],[], 'r--', lw=1.5, label="Beam center")[0]
    beam_left  = ax.plot([],[],[], 'k--', lw=1.0, label="Beam contour")[0]
    beam_right = ax.plot([],[],[], 'k--', lw=1.0)[0]
    traj_line  = ax.plot([],[],[], 'b-', lw=2.0, label="Trajectory")[0]
    sat_marker = ax.scatter([xs[0]],[ys[0]],[zs[0]], marker='^', s=120,
                            color='orange', edgecolors='k', linewidths=0.8,
                            label="Radar")
    virt       = ax.plot([],[],[], 'bx', markersize=8, linestyle='', label="Virtual elements")[0]

    ax.legend(loc='upper left', bbox_to_anchor=(1.05,1), fontsize=9)
    fig.tight_layout()

    # Prevent the figure from being auto-captured (critical for Voilà perf)
    plt.close(fig)

    # Front-end image that we update with PNG bytes each frame
    img_widget = WImage(format='png', layout=Layout(width=f"{IMG_WIDTH_PX}px"))

    def _render_png() -> bytes:
        buf = io.BytesIO()
        # savefig triggers a draw on non-interactive backends
        fig.savefig(buf, format='png', dpi=DPI)
        buf.seek(0)
        return buf.getvalue()

    # ---------- Pure frame update (no accumulation!) ----------
    def _update_frame(k: int):
        k = int(k)
        xk, yk, zk = xs[k], ys[k], zs[k]
        gp = ground_point

        # LOS + beam edges
        los_line.set_data([xk, gp[0]], [yk, gp[1]])
        los_line.set_3d_properties([zk, gp[2]])

        rng = np.sqrt((xk-gp[0])**2 + (yk-gp[1])**2 + (zk-gp[2])**2)
        hw  = np.tan(np.radians(BEAM_HALF_ANGLE_DEG)) * rng
        beam_left.set_data([xk, gp[0]-hw], [yk, gp[1]]);  beam_left.set_3d_properties([zk, gp[2]])
        beam_right.set_data([xk, gp[0]+hw], [yk, gp[1]]); beam_right.set_3d_properties([zk, gp[2]])

        # Trajectory up to k
        traj_line.set_data(xs[:k+1], ys[:k+1]); traj_line.set_3d_properties(zs[:k+1])

        # Move radar marker
        sat_marker._offsets3d = ([xk], [yk], [zk])

        # Virtual elements sampled up to k
        idx = np.arange(0, k+1, ELEMENT_SPACING)
        virt.set_data(xs[idx], ys[idx]); virt.set_3d_properties(zs[idx])

        # Push a fresh PNG to the frontend
        img_widget.value = _render_png()

    # ---------- Controls ----------
    play   = Play(interval=int(1000/FPS), min=0, max=FRAMES-1, step=1, value=0)  # ~10 s playback
    slider = IntSlider(value=0, min=0, max=FRAMES-1, step=1, description="Frame")
    jslink((play,'value'), (slider,'value'))

    reset_btn    = Button(description="Reset", layout=Layout(width='90px'))
    recenter_btn = Button(description="Recenter", layout=Layout(width='110px'))
    save_btn     = Button(description="Save PNG", layout=Layout(width='110px'))

    def _on_reset(_):
        slider.value = 0
        _update_frame(0)

    def _on_recenter(_):
        ax.set_xlim(*xlim); ax.set_ylim(*ylim); ax.set_zlim(*zlim)
        img_widget.value = _render_png()

    def _on_save(_):
        try:
            ts = datetime.now().strftime("%Y%m%d_%H%M%S")
            fname = f"spotlight_frame_{int(slider.value):04d}_{ts}.png"
            out_dir = os.path.join("outputs")
            os.makedirs(out_dir, exist_ok=True)
            with open(os.path.join(out_dir, fname), "wb") as f:
                f.write(_render_png())
        except Exception as e:
            pass

    slider.observe(lambda ch: _update_frame(ch['new']), names='value')
    reset_btn.on_click(_on_reset)
    recenter_btn.on_click(_on_recenter)
    save_btn.on_click(_on_save)

    # ---------- Initial frame ----------
    _update_frame(0)

    # ---------- Layout ----------
    header = Label("SPOTLIGHT (10 s fixed) with Bat outline on ground")
    return VBox([header, HBox([play, slider, reset_btn, recenter_btn, save_btn], layout=Layout(align_items='center', column_gap='10px')), img_widget])

sar_collection_simulation = build_spotlight_with_bat_outline_fixed_app()
display(sar_collection_simulation)  # Display the UI in Jupyter/Voilà

## 📡 SAR data

In our spotlight example, we are illuminating the *same patch of ground* throughout the whole measurement. That means every time we transmit a pulse, we see echoes from the same targets — only their **ranges** change as the platform moves.  

👉 In the **raw data**, we cannot yet separate targets along the flight path (the azimuth direction). All we see is how their slant ranges vary pulse by pulse.  

But here’s the clever trick: **beamforming on receive**. 🎯 Remember in the earlier notebooks how we used the floating buoy array to figure out the **direction of incoming waves** by steering our receive beam? 

SAR is based on **exactly the same principle** — except our “array” is virtual. Since we’ve collected echoes all along the trajectory, we can treat the measurements along the flight path as **one enormous antenna array**! By carefully combining the measurements with the correct phases, we can form an ultra-narrow receive beam and unlock fine azimuth resolution. ✨  

Now, as we explored in the previous notebook, this **synthetic aperture** magic ✨ is combined with the use of **chirped signals** 🎵 and **matched filtering** 🎯 to achieve high range resolution.  

Before we fuse the echoes together to form our razor-thin azimuth beam 📡➡️📍, we first apply matched filtering along the range direction. This step sharpens the echoes into **ultra-crisp, ultra-powerful pulses** ⚡🔍 — giving us the fine detail needed to build the SAR image.  

Below, you can play with a simulation of the **range-compressed data** from a simulated ground target shape 🦇. Here the transmitted chirp has already been processed to give us high **range resolution** (remember the last notebook!). As you increase the bandwidth, the signatures in range get sharper and narrower.  

And watch closely 🔍: because the radar moves, the **distance to each target shifts from one measurement to the next**. This shows up as **curved traces** across the pulses — almost like bright, banana-shaped streaks 🍌 winding their way through the data matrix. The longer the collection, the more the range changes, and the curvier the bananas. These curvy lines are the raw fingerprints of our targets, waiting to be focused into a crisp SAR image.


In [ ]:
def build_bat_rangecmp_voila_app():
    """
    Miniature SAR 'range-compressed' simulator (all in METERS).
    Fast Voila version:
      - Sparse windowed sinc accumulation per target (no giant kernel arrays)
      - Server-side Matplotlib -> PNG bytes -> ipywidgets.Image
    Adjustable:
      - Bandwidth [MHz] -> ΔR = c/(2B)
      - Duration [s]    -> aperture length = v * duration
    """
    import io
    import os
    from datetime import datetime
    import math
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import VBox, HBox, FloatSlider, Button, Layout, Image as WImage, Label, Checkbox, Dropdown
    from IPython.display import display

    # -------------------------
    # Constants
    # -------------------------
    c = 299_792_458.0  # m/s
    ALTITUDE_M      = 500.0
    INCIDENCE_DEG   = 30.0
    VELOCITY_MPS    = 7.5
    WAVELENGTH_M    = 0.10
    RANGE_WINDOW_M  = 200.0
    BAT_SCALE_M     = 40.0
    FRAMES_NPULSES  = 100
    BAT_SUBSAMPLE   = 2
    # sinc support half-width in samples (total window = 2*W+1)
    KERNEL_HALFW    = 12
    DPI             = 110

    # -------------------------
    # Bat outline (unit coords -> scaled meters)
    # -------------------------
    def _make_bat_points():
        X = np.zeros((0))
        Y = np.arange(-4, 4, 0.1)
        for y in Y:
            X = np.append(X, abs(y/2) - 0.09137*y**2 + math.sqrt(1 - (abs(abs(y) - 2) - 1)**2) - 3)

        Y1 = np.append(np.arange(-7, -3, 0.1), np.arange(3, 7, 0.1))
        X1 = [3*math.sqrt(1 - (yy/7)**2) for yy in Y1]
        X = np.append(X, X1); Y = np.append(Y, Y1)

        Y1 = np.append(np.arange(-7, -4, 0.1), np.arange(4, 7, 0.1))
        X1 = [-3*math.sqrt(1 - (yy/7)**2) for yy in Y1]
        X = np.append(X, X1); Y = np.append(Y, Y1)

        Y1 = np.append(np.arange(-1, -0.8, 0.1), np.arange(0.8, 1, 0.1))
        X1 = [9 - 8*abs(yy) for yy in Y1]
        X = np.append(X, X1); Y = np.append(Y, Y1)

        Y1 = np.arange(-0.5, 0.5, 0.1)
        X1 = [2 for _ in Y1]
        X = np.append(X, X1); Y = np.append(Y, Y1)

        Y1 = np.append(np.arange(-2.9, -1, 0.1), np.arange(1, 2.9, 0.1))
        X1 = [1.5 - 0.5*abs(yy) - 1.89736*(math.sqrt(3 - yy**2 + 2*abs(yy)) - 2) for yy in Y1]
        X = np.append(X, X1); Y = np.append(Y, Y1)

        Y1 = np.append(np.arange(-0.7, -0.45, 0.05), np.arange(0.45, 0.7, 0.05))
        X1 = [3*abs(yy)+0.75 for yy in Y1]
        X = np.append(X, X1); Y = np.append(Y, Y1)

        X = -X
        return np.asarray(X), np.asarray(Y)

    Xbat_u, Ybat_u = _make_bat_points()
    bat_x0 = (Xbat_u * BAT_SCALE_M)[::BAT_SUBSAMPLE].astype(np.float32)
    bat_y0 = (Ybat_u * BAT_SCALE_M)[::BAT_SUBSAMPLE].astype(np.float32)
    bat_z0 = np.zeros_like(bat_x0, dtype=np.float32)
    targets0 = np.column_stack([bat_x0, bat_y0, bat_z0])  # (Nt, 3)

    # -------------------------
    # Widgets
    # -------------------------
    w_bw_mhz = FloatSlider(value=300.0, min=10.0, max=1200.0, step=5.0,
                           description="Bandwidth [MHz]", continuous_update=False)
    w_bw_mhz.style.description_width = 'initial'
    w_dur_s  = FloatSlider(value=10.0, min=0.5, max=25.0, step=0.5,
                           description="Duration [s]", continuous_update=False)
    w_dur_s.style.description_width = 'initial'
    w_hires  = Checkbox(value=False, description="High-res range grid (caps at 4096)")
    w_hires.style.description_width = 'initial'
    run_btn  = Button(description="Run simulation", button_style="primary",
                      layout=Layout(width="180px"))
    save_btn = Button(description="Save PNG", layout=Layout(width="110px"))
    reset_btn= Button(description="Reset", layout=Layout(width="90px"))
    readout  = Label()

    # Front-end image widget (we stream PNGs into this)
    img = WImage(format='png', layout=Layout(width="820px", height="460px"))

    # Colormap dropdown (define early so it can be used in simulate_and_render)
    cmap_dd = Dropdown(options=['jet', 'viridis', 'plasma', 'gray', 'coolwarm'], 
                       value='gray', description='Colormap:', 
                       style={'description_width':'initial'})

    # -------------------------
    # Core simulation (sparse)
    # -------------------------
    def simulate_and_render():
        B = float(w_bw_mhz.value) * 1e6
        T = float(w_dur_s.value)
        v = VELOCITY_MPS
        H = ALTITUDE_M
        inc = np.radians(INCIDENCE_DEG)

        # Geometry
        y_pos = H * np.tan(inc)
        x_radar = np.linspace(-v*T/2, v*T/2, FRAMES_NPULSES, dtype=np.float32)
        y_radar = np.full_like(x_radar, y_pos, dtype=np.float32)
        z_radar = np.full_like(x_radar, H, dtype=np.float32)
        r0 = np.sqrt(y_pos**2 + H**2).astype(np.float32)

        # Range axis
        dr = (c / (2.0*B)).astype(np.float32) if isinstance(B, np.ndarray) else np.float32(c / (2.0*B))
        # Base size; cap to keep Voila smooth
        base_n = int(np.clip(4*round(RANGE_WINDOW_M / float(dr)), 512, 4096 if w_hires.value else 2048))
        range_axis = (np.linspace(-RANGE_WINDOW_M/2, RANGE_WINDOW_M/2, base_n, dtype=np.float32) + r0).astype(np.float32)

        # Allocate result
        rngcmp = np.zeros((FRAMES_NPULSES, base_n), dtype=np.complex64)

        # Precompute constant factor
        two_over_lambda = np.float32(2.0 / WAVELENGTH_M)
        W = KERNEL_HALFW

        # Sparse, windowed accumulation
        targets = targets0  # (Nt,3) float32
        Nt = targets.shape[0]
        for i in range(FRAMES_NPULSES):
            # radar position
            rp_x, rp_y, rp_z = x_radar[i], y_radar[i], z_radar[i]
            # distances to all targets (float32)
            dx = targets[:,0] - rp_x
            dy = targets[:,1] - rp_y
            dz = targets[:,2] - rp_z
            dists = np.sqrt(dx*dx + dy*dy + dz*dz).astype(np.float32)       # (Nt,)

            # nearest bin for each target
            idx_float = (dists - range_axis[0]) / (range_axis[1] - range_axis[0])
            idx0 = np.clip(idx_float.astype(np.int32), 0, base_n-1)

            # complex phasor per target (constant across range for that target)
            ph = np.exp(1j * 2.0 * np.pi * two_over_lambda * dists).astype(np.complex64)  # (Nt,)

            # accumulate into small windows
            for t in range(Nt):
                j0 = idx0[t]
                j1 = max(0, j0 - W)
                j2 = min(base_n, j0 + W + 1)
                # local bins
                rr = range_axis[j1:j2]
                # small sinc kernel around the target peak (float32)
                k_local = np.sinc((rr - dists[t]) / dr).astype(np.float32)   # (<= 2W+1,)
                rngcmp[i, j1:j2] += ph[t] * k_local  # complex add

        # -------------------------
        # Render to PNG
        # -------------------------
        fig, ax = plt.subplots(figsize=(8.2, 4.2), dpi=DPI)
        im = ax.imshow(np.abs(rngcmp),
                       aspect='auto',
                       cmap=cmap_dd.value,
                       extent=[float(range_axis[0]), float(range_axis[-1]), 0.0, T])
        ax.invert_yaxis()
        ax.set_xlabel("Slant range (m)")
        ax.set_ylabel("Time (s)")
        ax.set_title("SAR data before azimuth beamforming")
        fig.colorbar(im, ax=ax, label="|echo|")
        fig.tight_layout()

        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=DPI)
        plt.close(fig)
        buf.seek(0)
        img.value = buf.getvalue()

        readout.value = f"ΔR ≈ {float(dr):.3f} m  |  pulses = {FRAMES_NPULSES}  |  range bins = {base_n}"

    # -------------------------
    # Wire up UI
    # -------------------------
    def _run(_=None):
        simulate_and_render()

    def _on_save(_=None):
        try:
            ts = datetime.now().strftime("%Y%m%d_%H%M%S")
            fname = f"rangecmp_{int(w_bw_mhz.value)}MHz_{float(w_dur_s.value):.1f}s_{ts}.png"
            out_dir = os.path.join("outputs")
            os.makedirs(out_dir, exist_ok=True)
            # Re-render current to ensure file matches display
            simulate_and_render()
            with open(os.path.join(out_dir, fname), "wb") as f:
                f.write(img.value)
        except Exception:
            pass

    def _on_reset(_=None):
        w_bw_mhz.value = 300.0
        w_dur_s.value = 10.0
        w_hires.value = False
        simulate_and_render()

    run_btn.on_click(_run)
    save_btn.on_click(_on_save)
    reset_btn.on_click(_on_reset)
    # Colormap observer
    def _update_colormap(_):
        simulate_and_render()
    cmap_dd.observe(_update_colormap, names='value')
    
    # initial draw
    simulate_and_render()
    
    controls = VBox([
        HBox([w_bw_mhz, w_dur_s, w_hires, cmap_dd],
             layout=Layout(align_items='center', column_gap='12px')),
        HBox([run_btn, reset_btn, save_btn],
             layout=Layout(align_items='center', column_gap='12px'))
    ])
    return VBox([controls, readout, img])

# ---- Build & display the Voila app ----
app = build_bat_rangecmp_voila_app()
display(app)

### 🧪 Try this (range‑compressed data)
- Increase **Bandwidth**: what property of the echo data changes?
- Increase **Duration**: what happens to the echo curves across time?


## 🖼️ SAR Processing

We’ve got our **SAR data** — now it’s time to focus it into a **sharp, high-resolution radar image**! ✨  

The magic of SAR imaging lies in **focusing the echoes** we collected along the flight path into a beam so narrow that it acts like a giant array. 🎯 To do this, we need to carefully account for the **changing distances** between the radar (at each point along the synthetic aperture) and every target in the scene. 📡📍  

Think back to how a real phased array can steer its beam by **applying different phase shifts** to its antenna elements. In SAR, we do exactly the same thing — except our “antenna elements” are the radar positions along the flight path. For any chosen focus point on the ground, we compute the precise distance from each virtual antenna position to that point. This tells us which samples to line up, and what **phase correction** to apply.  

At its core, SAR processing is beautifully simple:  
👉 It’s just **summing up the recorded echoes** again and again, each time with the correct phase shifts, to “steer” the synthetic beam toward different points in the scene.  

The SAR processor’s job is simple in principle: it **turns the beam** — or the “ears” 👂 of our synthetic array — toward every pixel position in the image.  

You’ve already seen how this works in the notebook on **receive beamforming** 🎯:  
- We take the echoes from each antenna element (here they are *virtual* ones 🪄),  
- Calculate how much each needs to be nudged in delay and phase 🔄,  
- And then add them up ➕.  

We repeat this process for **every single pixel** in the scene. The pixel value is nothing more than the **length of the phasor** we get from that beamforming calculation.  

To do this properly, two key pieces of information are needed:  
1. 🗺️ Which part of the ground was illuminated during collection  
2. 🛰️ Where the satellite was at each measurement position  

With these in hand, we can calculate the exact time delays ⏱️ and phases 🔀 of the echoes. Aligning them just right allows us to combine the signals **coherently** — producing those sharp, focused echoes that form a SAR image. ✨  

The resolving power of our image then follows directly from physics:  
- **Azimuth resolution** 🔄 is set by the **length of the synthetic aperture** — the longer the flight path we combine, the narrower the beam and the sharper the detail. For satellites, where the velocity is nearly constant, the **collection time** determines the aperture length, and thus our azimuth resolution.  
- **Range resolution** 📏 is governed by the **signal bandwidth**, just as we learned before with pulse compression.  

Below, you can experiment interactively with these two levers — bandwidth and duration — and see how they shape the final SAR image from our fictional collection. 🚀


In [ ]:
def build_bat_2d_sar_image_voila_app():
    """
    Voilà-ready 2D SAR toy imager (meters).
    Fast version:
      - Windowed separable accumulation: sinc(ΔR/ΔR) along y, sinc(Δx/Δx) along x
      - Server-side Matplotlib -> PNG bytes -> ipywidgets.Image
    Controls: Bandwidth [MHz], Duration [s], High-res toggle
    Fixed miniature params:
      wavelength = 0.10 m, altitude = 500 m, velocity = 7.5 m/s, incidence = 30 deg
      bat outline ~40 m across (z=0)
    """
    import io, math, os
    from datetime import datetime
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import VBox, HBox, FloatSlider, Button, Layout, HTML, Image as WImage, Checkbox, Label, Dropdown
    from IPython.display import display

    # ---- Constants (miniature geometry) ----
    c             = 299_792_458.0
    wavelength_m  = 0.10
    altitude_m    = 500.0
    velocity_mps  = 7.5
    incidence_deg = 30.0
    bat_scale_m   = 40.0
    subsample     = 2        # thin bat points for speed
    point_spacing = 0.075
    DPI           = 110
    KY_HALF       = 16       # half-width (samples) of range sinc window (total ~33)
    KX_HALF       = 16       # half-width (samples) of azimuth sinc window (total ~33)
    NX_MAX        = 2000     # caps to keep Voila responsive (hires raises to 2000)
    NY_MAX        = 2000

    # ---- Bat outline (z=0), scaled to meters ----
    def make_batsignal_points_scaled(scale_m=40.0, subsample=1):
        X = np.zeros((0))
        Y = np.arange(-4, 4, point_spacing)
        for y in Y:
            X = np.append(X, abs(y/2) - 0.09137*y**2 + math.sqrt(1 - (abs(abs(y) - 2) - 1)**2) - 3)
        Y1 = np.append(np.arange(-7, -3, point_spacing), np.arange(3, 7, point_spacing))
        X1 = np.array([3*np.sqrt(1 - (yy/7)**2) for yy in Y1]); X = np.append(X, X1); Y = np.append(Y, Y1)
        Y1 = np.append(np.arange(-7, -4, point_spacing), np.arange(4, 7, point_spacing))
        X1 = np.array([-3*np.sqrt(1 - (yy/7)**2) for yy in Y1]); X = np.append(X, X1); Y = np.append(Y, Y1)
        Y1 = np.append(np.arange(-1, -0.8, point_spacing), np.arange(0.8, 1, point_spacing))
        X1 = np.array([9 - 8*abs(yy) for yy in Y1]); X = np.append(X, X1); Y = np.append(Y, Y1)
        Y1 = np.arange(-0.5, 0.5, point_spacing)
        X1 = np.full_like(Y1, 2.0, dtype=float); X = np.append(X, X1); Y = np.append(Y, Y1)
        Y1 = np.append(np.arange(-2.9, -1, point_spacing), np.arange(1, 2.9, point_spacing))
        X1 = np.array([1.5 - 0.5*abs(yy) - 1.89736*(np.sqrt(3 - yy**2 + 2*abs(yy)) - 2) for yy in Y1])
        X = np.append(X, X1); Y = np.append(Y, Y1)
        Y1 = np.append(np.arange(-0.7, -0.45, 0.05), np.arange(0.45, 0.7, 0.05))
        X1 = np.array([3*abs(yy)+0.75 for yy in Y1]); X = np.append(X, X1); Y = np.append(Y, Y1)
        X = -X

        width_unit = X.max() - X.min()
        s = scale_m / width_unit if width_unit != 0 else 0.0
        X = X * s; Y = Y * s

        if subsample > 1:
            X = X[::subsample]; Y = Y[::subsample]
        Z = np.zeros_like(X)
        return X.astype(np.float32), Y.astype(np.float32), Z.astype(np.float32)

    bat_x, bat_y, bat_z = make_batsignal_points_scaled(bat_scale_m, subsample)

    # ---- Helpers ----
    sinc = np.sinc  # normalized: sin(pi x)/(pi x)

    # ---- UI ----
    w_bw  = FloatSlider(value=300, min=10, max=1200, step=5,
                        description="Bandwidth [MHz]", continuous_update=False)
    w_bw.style.description_width = 'initial'
    w_dur = FloatSlider(value=10.0, min=0.5, max=25.0, step=0.5,
                        description="Duration [s]", continuous_update=False)
    w_dur.style.description_width = 'initial'
    run_btn = Button(description="Render image", button_style="primary",
                     layout=Layout(width="160px"))
    save_btn = Button(description="Save PNG", layout=Layout(width="110px"))
    reset_btn= Button(description="Reset", layout=Layout(width="90px"))
    readout = Label()

    # Front-end image (PNG bytes pumped into this)
    img = WImage(format='png', layout=Layout(width="820px", height="520px"))

    # Colormap dropdown (define early so it can be used in render)
    cmap_dd = Dropdown(options=['jet', 'viridis', 'plasma', 'gray', 'coolwarm'], 
                       value='gray', description='Colormap:', 
                       style={'description_width':'initial'})

    def render(_=None):
        B = float(w_bw.value) * 1e6
        T = float(w_dur.value)

        # Resolutions
        dR   = c / (2.0 * B)                              # range resolution
        H    = altitude_m
        inc  = math.radians(incidence_deg)
        y_r  = H * math.tan(inc)                          # radar ground-range (mid-track)
        R0   = math.sqrt(y_r**2 + H**2)                   # slant range to scene center
        L    = velocity_mps * T
        theta= L / R0                                     # small-angle
        dx   = wavelength_m / (2.0 * theta) if theta > 0 else 1e9  # azimuth resolution

        # Grid bounds (pad beyond bat footprint)
        pad   = 0.25
        x_min, x_max = float(bat_x.min()), float(bat_x.max())
        y_min, y_max = float(bat_y.min()), float(bat_y.max())
        x_span, y_span = (x_max - x_min), (y_max - y_min)
        gx_min, gx_max = x_min - pad*x_span, x_max + pad*x_span
        gy_min, gy_max = y_min - pad*y_span, y_max + pad*y_span

        # Pixel size: sample a bit finer than the tighter resolution
        cell = max(0.10, min(dR, dx) / 2.5)
        Nx = int(np.clip(np.ceil((gx_max - gx_min) / cell), 256, NX_MAX))
        Ny = int(np.clip(np.ceil((gy_max - gy_min) / cell), 256, NY_MAX))

        x = np.linspace(gx_min, gx_max, Nx, dtype=np.float32)
        y = np.linspace(gy_min, gy_max, Ny, dtype=np.float32)

        # Precompute slant range vs y (x≈0 approximation for ΔR term)
        # Rt(y) = sqrt((y - y_r)^2 + H^2)
        Rt_y = np.sqrt((y - y_r)*(y - y_r) + H*H).astype(np.float32)

        # Base image
        img_arr = np.zeros((Ny, Nx), dtype=np.float32)

        # Window sizes in samples
        wy = max(3, KY_HALF)
        wx = max(3, KX_HALF)

        # Accumulate outer products per target into local windows
        for xt, yt in zip(bat_x, bat_y):
            Rt_target = math.sqrt((yt - y_r)**2 + H**2)

            # y-window indices
            # find nearest y index to target y, then window
            jy0 = int(np.clip(np.searchsorted(y, yt), 0, Ny-1))
            jy1 = max(0, jy0 - wy)
            jy2 = min(Ny, jy0 + wy + 1)

            # x-window indices
            jx0 = int(np.clip(np.searchsorted(x, xt), 0, Nx-1))
            jx1 = max(0, jx0 - wx)
            jx2 = min(Nx, jx0 + wx + 1)

            # local kernels
            dR_row = Rt_y[jy1:jy2] - Rt_target           # (ny_local,)
            ky = sinc(dR_row / dR).astype(np.float32)

            dx_col = x[jx1:jx2] - xt                     # (nx_local,)
            kx = sinc(dx_col / dx).astype(np.float32)

            # outer product add
            img_arr[jy1:jy2, jx1:jx2] += np.outer(ky, kx)

        # Normalize for display
        vmax = float(img_arr.max()) if img_arr.size else 1.0
        if vmax > 0:
            img_disp = img_arr / vmax
        else:
            img_disp = img_arr

        # ----- Render to PNG bytes -----
        fig, ax = plt.subplots(figsize=(8.2, 5.6), dpi=DPI)
        imshow = ax.imshow(img_disp, origin='lower',
                                                     extent=[float(x.min()), float(x.max()), float(y.min()), float(y.max())],
                          aspect='auto', cmap=cmap_dd.value)
        ax.set_title("2D SAR Image (toy model)")
        ax.set_xlabel("Azimuth x (m)")
        ax.set_ylabel("Ground-range y (m)")
        fig.colorbar(imshow, ax=ax, label="Amplitude (norm.)")
        fig.tight_layout()

        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=DPI)
        plt.close(fig)
        buf.seek(0)
        img.value = buf.getvalue()

        readout.value = (f"Range res. ≈ {dR:.2f} m | Azimuth res. ≈ {dx:.2f} m | "
                         f"grid {Ny}×{Nx} px")

    # initial render + wire up
    def _on_save(_=None):
        try:
            ts = datetime.now().strftime("%Y%m%d_%H%M%S")
            fname = f"sar_image_{int(w_bw.value)}MHz_{float(w_dur.value):.1f}s_{ts}.png"
            out_dir = os.path.join("outputs")
            os.makedirs(out_dir, exist_ok=True)
            render()
            with open(os.path.join(out_dir, fname), "wb") as f:
                f.write(img.value)
        except Exception:
            pass

    def _on_reset(_=None):
        w_bw.value = 300
        w_dur.value = 10.0
        render()

    # Colormap observer
    def _update_colormap(_):
        render()
    cmap_dd.observe(_update_colormap, names='value')
    
    run_btn.on_click(render)
    save_btn.on_click(_on_save)
    reset_btn.on_click(_on_reset)
    render()

    header = HTML("<b>Focused SAR Image</b>")
    controls = VBox([
        HBox([w_bw, w_dur, cmap_dd],
             layout=Layout(align_items='center', column_gap='12px')),
        HBox([run_btn, reset_btn, save_btn],
             layout=Layout(align_items='center', column_gap='12px'))
    ])
    return VBox([header, controls, readout, img])

# Build & display in Jupyter/Voilà
app_2d = build_bat_2d_sar_image_voila_app()
display(app_2d)

### 🧪 Try this (SAR image)
- Increase **Duration**: how does the image change along **azimuth x**?
- Increase **Bandwidth**: how does the image change along **ground‑range y**?

## ✨ SAR Magic 🪄 

SAR works by **cleverly combining measured signals** through digital computation 💻 after the data has been collected.  

Why do we go through all this effort? 🤔 Because hardware alone has its limits:  
- We cannot build antennas that are kilometers long 📡❌  
- Nor can we transmit ultra-short, ultra-powerful pulses ⚡❌  

But by applying smart **signal processing** 🎯, we can create these capabilities *virtually*:  
- 🛰️ A giant synthetic antenna built from many measurement points  
- 🎵 Ultra-sharp pulses obtained through chirping and matched filtering  

In other words: SAR turns physical limitations into opportunities — achieving resolution and detail that would be impossible with hardware alone. 🚀  

## 📌 Summary
- **Bandwidth → range resolution**: larger bandwidth sharpens features along range.
- **Aperture length → azimuth resolution**: longer collection narrows the beam and sharpens along azimuth.
- **Synthetic aperture = phantom array**: combine echoes from many positions to focus like a giant antenna.